# Persistent enrichment of multidrug resistant Klebsiella in oral and nasal communities during long-term starvation

Jupyter Notebook containing Python code used for analyses featured in "Persistent enrichment of multidrug resistant Klebsiella in oral and nasal communities during long-term starvation" (2023). 
  
Should be run after R code.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import urllib.request
import os
import glob
from scipy.stats import ttest_ind
from statannotations.Annotator import Annotator
from statistics import mean
import matplotlib as mpl


In [ ]:
#make figure text editable
new_rc_params = {'text.usetex': False,
"svg.fonttype": 'none'
}
plt.rcParams.update(new_rc_params)

In [ ]:
#set working directory (this should be the same output directory as the one used for the R code)
working_dir = "/home/ubuntu/data/Projects/Nell_Saliva_Nares/publication_run/"

#set output directory (this should be the same output directory as the one used for the R code)
output_dir = "/home/ubuntu/data/Projects/Nell_Saliva_Nares/publication_run/output/"

### compare genome size to starvation enrichment using eHOMD genomes

In [ ]:
SEQID_info_df = pd.read_csv(f"{working_dir}required_files/SEQID_info_parsed_one_representative_per_hmt.csv")

## Species level abundance bar plots

In [ ]:
#format species_level_abundance
species_level_abund_df = pd.read_table(f"{output_dir}relative_abundance.species.tsv", sep="\t")


In [ ]:
species_level_abund_df = species_level_abund_df.set_index('genus_species_hmt') * 100
species_level_abund_df = species_level_abund_df.T.reset_index().rename(columns={'index': "sample_name"})

species_list = list(species_level_abund_df.drop(columns="sample_name").columns)

sample_metadata_df = pd.read_csv(f"{working_dir}required_files/sample_group_metadata.csv")

species_level_abund_df = sample_metadata_df.merge(species_level_abund_df, on="sample_name", how="left")

### individual bar plots

nares raw vs post

In [ ]:
#nares raw vs nares post
species_level_abund_df_nares_raw_nares_post = species_level_abund_df[species_level_abund_df.Saliva_nares == "Nares"][species_level_abund_df.condition.isin(["Raw_sample", "shi_media_Post_starvation"])]

### longform delta strip plots

In [ ]:
species_level_abund_df

In [ ]:
# make master df 
def make_species_raw_post_df(species_level_abundance_df):
    species_raw_post_abun_delta_dict = defaultdict(list)

    #species_occurance_dict
    species2occurance = {}

    #create df
    for species in species_list:

        occurance_count = 0
        #grab the change in relative abundance from raw to post, discard samples that do not have the genera in the raw sample or the post_starve sample

        for sample_num in species_level_abundance_df.Sample_num.unique():

            #raw sample abundance
            raw_num = species_level_abundance_df[species_level_abundance_df.condition == "Raw_sample"][species_level_abundance_df.Sample_num == sample_num][species].reset_index(drop=True)[0]
            #post_starve abundance
            post_starve_num = species_level_abundance_df[species_level_abundance_df.condition == "shi_media_Post_starvation"][species_level_abundance_df.Sample_num == sample_num][species].reset_index(drop=True)[0]
            #difference
            delta = post_starve_num - raw_num

            #remove species that did not show up in either raw or post
            if ((raw_num == 0) and (post_starve_num == 0)):
                pass
            #do not consider the unclassifed asvs:
            elif species == "unclassified":
                pass
            else:
                #DF Information
                species_raw_post_abun_delta_dict["sample_num"].append(int(sample_num))
                species_raw_post_abun_delta_dict["species"].append(species)
                species_raw_post_abun_delta_dict["hmt_num"].append(int(species.rsplit("_",1)[1]))
                species_raw_post_abun_delta_dict["post_starve_abundance"].append(post_starve_num)
                species_raw_post_abun_delta_dict["raw_abundance"].append(raw_num)
                species_raw_post_abun_delta_dict["delta"].append(delta)

                #species occurance
                occurance_count += 1
        
        species2occurance[species] = occurance_count

    species_raw_post_abun_delta_df = pd.DataFrame(species_raw_post_abun_delta_dict)
    species_raw_post_abun_delta_df["occurance"] = species_raw_post_abun_delta_df.species.map(species2occurance)

    #grab mean abundance and mean delta
    species_2_mean_abundance = {}
    species_2_mean_abundance_raw = {}
    species_2_median_abundance = {}
    species_2_mean_delta = {}
    for species in species_raw_post_abun_delta_df.species.unique():
        sub_df = species_raw_post_abun_delta_df[species_raw_post_abun_delta_df.species == species]
        species_2_mean_abundance[species] = np.mean(sub_df.post_starve_abundance)
        species_2_mean_abundance_raw[species] = np.mean(sub_df.raw_abundance)
        species_2_mean_delta[species] = np.mean(sub_df.delta)
        species_2_median_abundance[species] = np.median(sub_df.post_starve_abundance)

    #map
    species_raw_post_abun_delta_df["mean_abun_post_starve"] = species_raw_post_abun_delta_df["species"].map(species_2_mean_abundance)
    species_raw_post_abun_delta_df["mean_abun_raw"] = species_raw_post_abun_delta_df["species"].map(species_2_mean_abundance_raw)
    species_raw_post_abun_delta_df["median_abun_post_starve"] = species_raw_post_abun_delta_df["species"].map(species_2_median_abundance)
    species_raw_post_abun_delta_df["mean_delta"] = species_raw_post_abun_delta_df["species"].map(species_2_mean_delta)

    return species_raw_post_abun_delta_df

nares raw vs nares post

In [ ]:
species_nares_raw_post_abun_delta_df = make_species_raw_post_df(species_level_abund_df_nares_raw_nares_post)

In [ ]:
#species_abundance across the samples, raw

sns.set(rc={'figure.figsize':(8,2)})

#open circle markers
pnts = np.linspace(0, np.pi * 2, 24)
circ = np.c_[np.sin(pnts) / 2, -np.cos(pnts) / 2]
vert = np.r_[circ, circ[::-1] * .7]
open_circle = mpl.path.Path(vert)

sns.set_style("ticks")
sns.despine()

ax = sns.stripplot(data=species_nares_raw_post_abun_delta_df, y="raw_abundance",marker=open_circle, x="sample_num", color='black', jitter =0, zorder=3)

#highlight samples with 50% abun of one species
for sample_number in set(species_nares_raw_post_abun_delta_df[species_nares_raw_post_abun_delta_df.raw_abundance >= 50].sample_num):
    plt.axvspan(int(sample_number)-1.5, int(sample_number)-0.5, color='#EF9A9A', alpha=0.3, lw=0)

#plt.axvspan(-0.5, 0.5, color='#EF9A9A', alpha=0.5)


plt.xlabel('')
plt.ylabel("relative abundance after starvation (%)")
plt.xticks(rotation=90)
plt.ylim(0, 104)
plt.xlim(-0.5, 30)

sns.despine()
plt.savefig(f"{output_dir}nares_raw_sample_num_rel_abun.svg", bbox_inches='tight')

In [ ]:
#species_abundance across the samples, post starve
sns.set(rc={'figure.figsize':(8,2)})

#open circle markers
pnts = np.linspace(0, np.pi * 2, 24)
circ = np.c_[np.sin(pnts) / 2, -np.cos(pnts) / 2]
vert = np.r_[circ, circ[::-1] * .7]
open_circle = mpl.path.Path(vert)

sns.set_style("ticks")
sns.despine()

ax = sns.stripplot(data=species_nares_raw_post_abun_delta_df, y="post_starve_abundance",marker=open_circle, x="sample_num", color='black', jitter =0, zorder=3)

#highlight samples with 50% abun of one species
for sample_number in set(species_nares_raw_post_abun_delta_df[species_nares_raw_post_abun_delta_df.post_starve_abundance >= 50].sample_num):
    plt.axvspan(int(sample_number)-1.5, int(sample_number)-0.5, color='#EF9A9A', alpha=0.3, lw=0)

plt.xlabel('')
plt.ylabel("relative abundance after starvation (%)")
plt.xticks(rotation=90)
plt.ylim(0, 104)
plt.xlim(-0.5, 30)

sns.despine()
plt.savefig(f"{output_dir}nares_post_sample_num_rel_abun.svg", bbox_inches='tight')

In [ ]:
nares_species_to_keep = []
#grab species with post_starve_abundance > 50
for species in species_nares_raw_post_abun_delta_df[species_nares_raw_post_abun_delta_df.post_starve_abundance >= 50].sort_values(by="median_abun_post_starve", ascending=False).species.unique():
    nares_species_to_keep.append(species)

print(len(nares_species_to_keep))

print(nares_species_to_keep)

#filter the df
species_nares_raw_post_abun_delta_df_filtered = species_nares_raw_post_abun_delta_df[species_nares_raw_post_abun_delta_df.species.isin(nares_species_to_keep)]

#sort
species_nares_raw_post_abun_delta_df_filtered.species=pd.Categorical(species_nares_raw_post_abun_delta_df_filtered.species,categories=nares_species_to_keep)
species_nares_raw_post_abun_delta_df_filtered=species_nares_raw_post_abun_delta_df_filtered.sort_values('species')


In [ ]:
#above 5% post starve abundance species
sns.set(rc={'figure.figsize':(4,1.5)})

#open circle markers
pnts = np.linspace(0, np.pi * 2, 24)
circ = np.c_[np.sin(pnts) / 2, -np.cos(pnts) / 2]
vert = np.r_[circ, circ[::-1] * .7]
open_circle = mpl.path.Path(vert)

sns.set_style("ticks")
sns.despine()

ax = sns.stripplot(data=species_nares_raw_post_abun_delta_df_filtered, y="post_starve_abundance",marker=open_circle, x="species", color='black', zorder=3)

ax = sns.boxplot(data= species_nares_raw_post_abun_delta_df_filtered, y="post_starve_abundance", x="species",
    whiskerprops={'visible': False}, 
    showfliers=False, 
    showbox=False,
    showcaps=False, 
    linewidth=0.8,
    zorder=2,
    medianprops={'color': '#F44336', 'ls': '-', 'lw': 2})

plt.xlabel('')
plt.ylabel("relative abundance after starvation (%)")
plt.xticks(rotation=90)
plt.ylim(0, 101)
sns.despine()
plt.savefig(f"{output_dir}nares_post_above_50_rel_abun.svg", bbox_inches='tight')

In [ ]:
#above 5% post starve abundance species delta
sns.set(rc={'figure.figsize':(4,1.5)})

#open circle markers
pnts = np.linspace(0, np.pi * 2, 24)
circ = np.c_[np.sin(pnts) / 2, -np.cos(pnts) / 2]
vert = np.r_[circ, circ[::-1] * .7]
open_circle = mpl.path.Path(vert)

sns.set_style("ticks")
sns.despine()

ax = sns.stripplot(data=species_nares_raw_post_abun_delta_df_filtered, y="delta",marker=open_circle, x="species", color='black', zorder=3)

ax = sns.boxplot(data= species_nares_raw_post_abun_delta_df_filtered, y="delta", x="species",
    whiskerprops={'visible': False}, 
    showfliers=False, 
    showbox=False,
    showcaps=False, 
    linewidth=0.8,
    zorder=2,
    medianprops={'color': '#F44336', 'ls': '-', 'lw': 2})

#vertical line
ax.axhline(0, ls='--', color="#B0BEC5", zorder=1)


plt.xlabel('')
plt.ylabel("change in relative abundance after starvation (%)")
plt.xticks(rotation=90)
plt.ylim(-max(abs(species_nares_raw_post_abun_delta_df_filtered["post_starve_abundance"]))-2, max(abs(species_nares_raw_post_abun_delta_df_filtered["post_starve_abundance"]))+2)
sns.despine()
plt.savefig(f"{output_dir}nares_post_above_50_delta.svg", bbox_inches='tight')

Saliva

In [ ]:
#saliva raw vs saliva post
species_level_abund_df_saliva_raw_post = species_level_abund_df[species_level_abund_df.Saliva_nares == "Saliva"][species_level_abund_df.condition.isin(["Raw_sample", "shi_media_Post_starvation"])]

In [ ]:
species_saliva_raw_post_abun_delta_df = make_species_raw_post_df(species_level_abund_df_saliva_raw_post)

In [ ]:
#species_abundance across the samples, raw

sns.set(rc={'figure.figsize':(8,2)})

#open circle markers
pnts = np.linspace(0, np.pi * 2, 24)
circ = np.c_[np.sin(pnts) / 2, -np.cos(pnts) / 2]
vert = np.r_[circ, circ[::-1] * .7]
open_circle = mpl.path.Path(vert)

sns.set_style("ticks")
sns.despine()

ax = sns.stripplot(data=species_saliva_raw_post_abun_delta_df, y="raw_abundance",marker=open_circle, x="sample_num", color='black', jitter =0, zorder=3)

#highlight samples with 50% abun of one species
for sample_number in set(species_saliva_raw_post_abun_delta_df[species_saliva_raw_post_abun_delta_df.raw_abundance >= 50].sample_num):
    plt.axvspan(int(sample_number)-1.5, int(sample_number)-0.5, color='#EF9A9A', alpha=0.3, lw=0)

#plt.axvspan(-0.5, 0.5, color='#EF9A9A', alpha=0.5)


plt.xlabel('')
plt.ylabel("relative abundance after starvation (%)")
plt.xticks(rotation=90)
plt.ylim(0, 104)
plt.xlim(-0.5, 30)

sns.despine()
plt.savefig(f"{output_dir}saliva_raw_sample_num_rel_abun.svg", bbox_inches='tight')

In [ ]:
#species_abundance across the samples, post starve
sns.set(rc={'figure.figsize':(8,2)})

#open circle markers
pnts = np.linspace(0, np.pi * 2, 24)
circ = np.c_[np.sin(pnts) / 2, -np.cos(pnts) / 2]
vert = np.r_[circ, circ[::-1] * .7]
open_circle = mpl.path.Path(vert)

sns.set_style("ticks")
sns.despine()

ax = sns.stripplot(data=species_saliva_raw_post_abun_delta_df, y="post_starve_abundance",marker=open_circle, x="sample_num", color='black', jitter =0, zorder=3)

#highlight samples with 50% abun of one species
for sample_number in set(species_saliva_raw_post_abun_delta_df[species_saliva_raw_post_abun_delta_df.post_starve_abundance >= 50].sample_num):
    plt.axvspan(int(sample_number)-1.5, int(sample_number)-0.5, color='#EF9A9A', alpha=0.3, lw=0)

plt.xlabel('')
plt.ylabel("relative abundance after starvation (%)")
plt.xticks(rotation=90)
plt.ylim(0, 104)
plt.xlim(-0.5, 30)

sns.despine()
plt.savefig(f"{output_dir}saliva_post_sample_num_rel_abun.svg", bbox_inches='tight')

In [ ]:
saliva_species_to_keep = []
#grab species with post_starve_abundance > 50
for species in species_saliva_raw_post_abun_delta_df[species_saliva_raw_post_abun_delta_df.post_starve_abundance >= 50].sort_values(by="median_abun_post_starve", ascending=False).species.unique():
    saliva_species_to_keep.append(species)

print(len(saliva_species_to_keep))

print(saliva_species_to_keep)

#filter the df
species_saliva_raw_post_abun_delta_df_filtered = species_saliva_raw_post_abun_delta_df[species_saliva_raw_post_abun_delta_df.species.isin(saliva_species_to_keep)]

#sort
species_saliva_raw_post_abun_delta_df_filtered.species=pd.Categorical(species_saliva_raw_post_abun_delta_df_filtered.species,categories=saliva_species_to_keep)
species_saliva_raw_post_abun_delta_df_filtered=species_saliva_raw_post_abun_delta_df_filtered.sort_values('species')


In [ ]:
#plot abun above 50
sns.set(rc={'figure.figsize':(0.5,1.5)})

#open circle markers
pnts = np.linspace(0, np.pi * 2, 24)
circ = np.c_[np.sin(pnts) / 2, -np.cos(pnts) / 2]
vert = np.r_[circ, circ[::-1] * .7]
open_circle = mpl.path.Path(vert)

sns.set_style("ticks")
sns.despine()

ax = sns.stripplot(data=species_saliva_raw_post_abun_delta_df_filtered, y="post_starve_abundance",marker=open_circle, x="species", color='black', zorder=3)

ax = sns.boxplot(data= species_saliva_raw_post_abun_delta_df_filtered, y="post_starve_abundance", x="species",
    whiskerprops={'visible': False}, 
    showfliers=False, 
    showbox=False,
    showcaps=False, 
    linewidth=0.8,
    zorder=2,
    medianprops={'color': '#F44336', 'ls': '-', 'lw': 2})

plt.xlabel('')
plt.ylabel("relative abundance after starvation (%)")
plt.xticks(rotation=90)
plt.ylim(0, max(abs(species_saliva_raw_post_abun_delta_df_filtered["post_starve_abundance"]))+2)
sns.despine()
plt.savefig(f"{output_dir}saliva_post_above_50_rel_abun.svg", bbox_inches='tight')

In [ ]:
#plot delta abund over 50
sns.set(rc={'figure.figsize':(0.5,1.5)})

#open circle markers
pnts = np.linspace(0, np.pi * 2, 24)
circ = np.c_[np.sin(pnts) / 2, -np.cos(pnts) / 2]
vert = np.r_[circ, circ[::-1] * .7]
open_circle = mpl.path.Path(vert)

sns.set_style("ticks")
sns.despine()

ax = sns.stripplot(data=species_saliva_raw_post_abun_delta_df_filtered, y="delta",marker=open_circle, x="species", color='black', zorder=3)

ax = sns.boxplot(data= species_saliva_raw_post_abun_delta_df_filtered, y="delta", x="species",
    whiskerprops={'visible': False}, 
    showfliers=False, 
    showbox=False,
    showcaps=False, 
    linewidth=0.8,
    zorder=2,
    medianprops={'color': '#F44336', 'ls': '-', 'lw': 2})

#vertical line
ax.axhline(0, ls='--', color="#B0BEC5", zorder=1)


plt.xlabel('')
plt.ylabel("change in relative abundance after starvation (%)")
plt.xticks(rotation=90)
plt.ylim(-max(abs(species_saliva_raw_post_abun_delta_df_filtered["delta"]))-4, max(abs(species_saliva_raw_post_abun_delta_df_filtered["delta"]))+4)
sns.despine()
plt.savefig(f"{output_dir}saliva_post_above_50_delta.svg", bbox_inches='tight')

## Genome size to CDS jackpot species

In [ ]:

nares_Post_enriched_species_hmts = [int(species.rsplit("_",1)[1]) for species in nares_species_to_keep]
saliva_Post_enriched_species_hmts = [int(species.rsplit("_",1)[1]) for species in saliva_species_to_keep]

In [ ]:
#species that went from above 1% abun in raw to less than 0.1 abun in post
nares_raw_species = list(species_nares_raw_post_abun_delta_df[species_nares_raw_post_abun_delta_df.mean_abun_raw >= 1][species_nares_raw_post_abun_delta_df.mean_abun_post_starve <= 0.1].species.apply(lambda x: int(x.rsplit("_",1)[1])).unique())

saliva_raw_species = list(species_saliva_raw_post_abun_delta_df[species_saliva_raw_post_abun_delta_df.mean_abun_raw >= 1][species_saliva_raw_post_abun_delta_df.mean_abun_post_starve <= 0.1].species.apply(lambda x: int(x.rsplit("_",1)[1])).unique())


In [ ]:
#map to seq info df
def map_raw_post_enriched(seqinfo_df, post_hmt_list, raw_hmt_list):
    seqinfo_df_copy = seqinfo_df.copy()
    seqinfo_df_copy["HMT_num"] = seqinfo_df_copy["HMT_ID"].apply(lambda x: int(x.rsplit("-",1)[1]))

    hmt2enrichment = {}
    for hmt_num in seqinfo_df_copy["HMT_num"].unique():
        if hmt_num in post_hmt_list:
            hmt2enrichment[hmt_num] = "post"
        elif hmt_num in raw_hmt_list:
            hmt2enrichment[hmt_num] = "raw"
        else:
            hmt2enrichment[hmt_num] = "neither"

    seqinfo_df_copy["raw_post"] = seqinfo_df_copy.HMT_num.map(hmt2enrichment)
    return(seqinfo_df_copy)

In [ ]:
SEQID_info_df_nares = map_raw_post_enriched(SEQID_info_df, nares_Post_enriched_species_hmts, nares_raw_species)
SEQID_info_df_saliva = map_raw_post_enriched(SEQID_info_df, saliva_Post_enriched_species_hmts, saliva_raw_species)

plot

In [ ]:
color_dict = {"raw":"#1565C0", "neither":"#CCD1D1", "post":"#FF5722"}

In [ ]:
#nares
sns.set(rc={'figure.figsize':(6,4)})
sns.set_style("ticks")

sns.scatterplot(data=SEQID_info_df_nares[SEQID_info_df_nares.raw_post == "neither"], x="Combined_Size", y="num_cds", hue="raw_post", palette=color_dict, alpha=0.8)
sns.scatterplot(data=SEQID_info_df_nares[SEQID_info_df_nares.raw_post == "raw"], x="Combined_Size", y="num_cds", hue="raw_post", palette=color_dict, alpha=0.8)
sns.scatterplot(data=SEQID_info_df_nares[SEQID_info_df_nares.raw_post == "post"], x="Combined_Size", y="num_cds", hue="raw_post", palette=color_dict, alpha=0.8)

plt.xlabel("Genome Size")
plt.ylabel("Number of CDS")


sns.despine()
plt.savefig(f"/{output_dir}nares_raw_post_genome_size_to_enrichment.svg", bbox_inches="tight")


In [ ]:
sns.set(rc={'figure.figsize':(4,4)})
sns.set_style("ticks")

sns.scatterplot(data=SEQID_info_df_saliva[SEQID_info_df_saliva.raw_post == "neither"], x="Combined_Size", y="num_cds", hue="raw_post", palette=color_dict, alpha=0.8)
sns.scatterplot(data=SEQID_info_df_saliva[SEQID_info_df_saliva.raw_post != "neither"], x="Combined_Size", y="num_cds", hue="raw_post", palette=color_dict, alpha=0.8)

plt.xlabel("Genome Size")
plt.ylabel("Number of CDS")

sns.despine()
plt.savefig(f"{output_dir}saliva_raw_post_genome_size_to_enrichment.svg", bbox_inches="tight")


## anvio functional enrichment of these genomes

In [ ]:
#make directory to download genomes
os.makedirs(f"{working_dir}ehomd_genomes", exist_ok=True)

In [ ]:
#download genomes
for key, row in SEQID_info_df_saliva[SEQID_info_df_saliva.raw_post != "neither"].iterrows():
    urllib.request.urlretrieve("https://www.homd.org/ftp/genomes/NCBI/V10.1/fna/{0}.fna".format(row["SEQ_ID"]), "{0}{1}.fna".format(working_dir+"ehomd_gneomes"),row["HMT_ID"].replace("-", "_")))

for key, row in SEQID_info_df_nares[SEQID_info_df_nares.raw_post != "neither"].iterrows():
    urllib.request.urlretrieve("https://www.homd.org/ftp/genomes/NCBI/V10.1/fna/{0}.fna".format(row["SEQ_ID"]), "{0}{1}.fna".format(working_dir + "ehomd_genomes",row["HMT_ID"].replace("-", "_")))

In [ ]:
#create table for anvio (nares samples)

ex_gen_df_nares = SEQID_info_df_nares[SEQID_info_df_nares.raw_post != "neither"][["HMT_ID", "raw_post"]].rename(columns={"HMT_ID": "name"})
ex_gen_df_nares["raw_post"] = "nares_" + ex_gen_df_nares["raw_post"]

ex_gen_df = ex_gen_df_nares

ex_gen_df["name"] = ex_gen_df["name"].str.replace("-", "_") + "_renamed"

ex_gen_df = ex_gen_df.drop_duplicates(subset="name")

ex_gen_df.to_csv(f"{working_dir}raw_post_df.tsv", sep="\t", index = False)

## functional enrichment (not pangenome functional enrichment)

In [ ]:
#create groups file
groups_df = pd.read_csv(f"{working_dir}raw_post_df.tsv", sep="\t")
groups_df = groups_df.rename(columns={"raw_post":"group"})
groups_df.to_csv(f"{working_dir}functional_enrichment_groups.txt", sep="\t", index=False)

## create enriched kegg module completeness heatmap

In [ ]:
kegg_functional_enrichment_module_df = pd.read_csv(f'{output_dir}functional_enrichment.txt', sep="\t")

#sort to p value less than 0.05 or equal
kegg_functional_enrichment_module_df = kegg_functional_enrichment_module_df[kegg_functional_enrichment_module_df.unadjusted_p_value <= 0.05].sort_values(by=["associated_groups", "adjusted_q_value"], ascending=[True, False]).reset_index(drop=True)

kegg_functional_enrichment_module_df["kegg_description_accession"] = kegg_functional_enrichment_module_df["KEGG_MODULE"] + "::" + kegg_functional_enrichment_module_df["accession"]

In [ ]:
hmtid2bacterianame = {}
for key, row in SEQID_info_df_nares.iterrows():
    hmtid2bacterianame[row["HMT_ID"]] = row["Genus"] + " " + row["Species"]

In [ ]:
#read through the estimate metabolism outputs

module_heatmap_dict = defaultdict(list)

for module_accession in kegg_functional_enrichment_module_df.kegg_description_accession:
    module_accession = module_accession.split(",",1)[0].split("::")[0] + "::" + module_accession.split("::",1)[1]
    module_heatmap_dict["kegg_description_accession"].append(module_accession)

for HMT_ID in SEQID_info_df_nares[SEQID_info_df_nares.raw_post !="neither"].sort_values(by=["raw_post", "Combined_Size"], ascending=[True,False])["HMT_ID"]:
    HMT_num = HMT_ID.rsplit("-", 1)[1]
    bacteria_name = hmtid2bacterianame[HMT_ID]
    estimated_metabolism_df = pd.read_table(f"output_dir/HMT_{0}_modules.txt".format(HMT_num), sep="\t").set_index("kegg_module", drop=True)

    for module_accession in kegg_functional_enrichment_module_df.accession:
        if module_accession in estimated_metabolism_df.index.values:
            completeness = estimated_metabolism_df.loc[module_accession]["module_completeness"] * 100
            module_heatmap_dict[bacteria_name].append(completeness)

        else:
            completeness = 0
            module_heatmap_dict[bacteria_name].append(completeness)

module_heatmap_df = pd.DataFrame(module_heatmap_dict)        
module_heatmap_df = module_heatmap_df.set_index("kegg_description_accession", drop=True)
module_heatmap_df = module_heatmap_df.replace(0, np.NaN)


In [ ]:
post_hmts = []
for HMT_ID in SEQID_info_df_nares[SEQID_info_df_nares.raw_post == "post"].sort_values(by=["Combined_Size"], ascending=[False])["HMT_ID"]:
    HMT_num = HMT_ID.rsplit("-", 1)[1]
    post_hmts.append(hmtid2bacterianame[HMT_ID])

raw_hmts = []
for HMT_ID in SEQID_info_df_nares[SEQID_info_df_nares.raw_post =="raw"].sort_values(by=["Combined_Size"], ascending=[False])["HMT_ID"]:
    HMT_num = HMT_ID.rsplit("-", 1)[1]
    raw_hmts.append(hmtid2bacterianame[HMT_ID])

module_heatmap_df_post = module_heatmap_df[post_hmts]
module_heatmap_df_raw = module_heatmap_df[raw_hmts]

In [ ]:
plt.figure(figsize=(5,12))
sns.heatmap(module_heatmap_df_post, xticklabels=1, yticklabels=1, linewidth=0.5, cmap=sns.color_palette("Blues", as_cmap=True), square=True)
plt.savefig("{output_dir}post_heatmap_blue.svg")
plt.show()
plt.close()

In [ ]:
plt.figure(figsize=(5,12))
sns.heatmap(module_heatmap_df_raw, xticklabels=1, yticklabels=1, linewidth=0.5, cmap=sns.color_palette("Blues", as_cmap=True), square=True)
plt.savefig(f"{output_dir}raw_heatmap.svg")
plt.show()
plt.close()